In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications.resnet import ResNet50

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = ResNet50(include_top=False, weights='imagenet', classifier_activation=None, classes=len(train.class_names),
                      pooling='average',input_shape=inputShape)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3) 

n_NeuronsPerFCLayers = []
n_Epochs = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = transferUtil.createTransferableModel(base_model, numberOfNeuronsPerFCLayer, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/ResnetTrials/resnet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 100:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if (devScore > 0.91) and (devScore > bestDevScore):
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_resnet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_resnet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],[adamLearningRate],
                                                                   [L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,
                                                                         L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
    
        possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
        adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_resnet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-09-22 19:22:11.523434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-09-22 19:22:11.523614: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-09-22 19:22:12.678608: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


redoing trial 0. Model was 111.40121459960938MB.


,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
0,25,9,0.000586,534.038881,NaN,NaN


trainScore
75/75 [==============================] - 97s 1s/step - loss: 3.4913 - f1_score: 0.0000e+00
devScore


...

,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
0,13,1,0.003712,0.897075,NaN,NaN


trainScore
75/75 [==============================] - 60s 785ms/step - loss: 0.7763 - f1_score: 0.8686
devScore
10/10 [==============================] - 8s 725ms/step - loss: 0.7904 - f1_score: 0.8374
concluding trial  6
trainScore
75/75 [==============================] - 61s 803ms/step - loss: 1.8401 - f1_score: 0.6494
devScore
10/10 [==============================] - 8s 720ms/step - loss: 1.8405 - f1_score: 0.6376
concluding trial  7
trainScore
75/75 [==============================] - 62s 806ms/step - loss: 0.6927 - f1_score: 0.0000e+00
devScore
10/10 [==============================] - 8s 740ms/step - loss: 0.6918 - f1_score: 0.0000e+00
concluding trial  8


,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,trainScore,devScore
3,11,9,0.000683,6.043446,0.937411,0.914603
6,4,1,0.000367,0.715549,0.868562,0.837441
1,7,5,0.008312,0.070790,0.720186,0.758372
7,10,6,0.002422,276.621112,0.649392,0.637643
2,5,7,0.002631,19.131644,0.562253,0.558079
0,4,1,0.008603,0.100181,0.000000,0.000000
4,7,8,0.002382,0.057079,0.000000,0.000000
5,3,3,0.008690,0.296267,0.000000,0.000000
8,1,6,0.002096,0.221563,0.000000,0.000000


Final Model


,n_NeuronsPerFCLayers,n_Epochs,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,11,9,0.000683,6.043446,99.58807,0.937411,0.914603,0.939833
